In [96]:
import datetime
import calendar
import numpy as np
import pandas as pd

from pyomo.environ import *
from pyomo.opt import SolverFactory
from datetime import date
from consumption import get_monthly_num_days_pct, annual_consumption
from calendar import monthrange, isleap

In [97]:
# Create a solver based on Gnu Linear Programming Kit (glpk)
opt = SolverFactory("glpk")
infinity = float('inf')

# Create an abstract model
model = AbstractModel()

In [98]:
##############################################
# Sets
##############################################
# http://pyomo.readthedocs.io/en/latest/data/datfiles.html

# Months
model.m = Set()

# Hours
model.h = Set()

# Days of Week
model.dw = Set()

# Date
model.d = Set()

# Indices for consumption
model.cons_ind = Set(model.m, model.h, model.dw)

In [99]:
##############################################
# Variables
##############################################

# Recommended system capacity
model.x = Var(model.m, within=NonNegativeReals)

In [100]:
##############################################
# Params
##############################################

# Costs
# param fixed_pc := 500
# param var_pc_solar_mod := 1
# param var_pc_inverter := 0.25
# param var_pc_electrical := 0.25
# param var_pc_structural := 0.1

model.fixed_pc = Param(default=500)
model.var_pc_solar_mod = Param(default=1) # per Watt
model.var_pc_inverter = Param(default=0.25)  # per Watt
model.var_pc_electrical = Param(default=0.25) # per Watt
model.var_pc_structural = Param(default=0.1) # per Watt
model.tou = Param(default=0.000162) # per Watt
model.nem = Param(default=0.000140) # per Watt

# Consumption(dates, hours)
model.cons = Param(model.d, model.h, within=NonNegativeReals, default=0.0)
# Minimum daily consumption
model.min_cons = Param(model.d, within=NonNegativeReals, default=0.0)
# Maximum daily consumption
model.max_cons = Param(model.d, within=NonNegativeReals, default=infinity)

# Irradiance(dates, hours): used to calculate generation
model.spi = Param(model.d, model.h, within=NonNegativeReals, default=0.0)
# Minimum daily spi
model.min_spi = Param(model.d, within=NonNegativeReals, default=0.0)
# Maximum daily spi
model.max_gen = Param(model.d, within=NonNegativeReals, default=1.0)

# # Minimum system capacity
# model.min_syscap = Param(model.m, within=NonNegativeReals, default=1000.0)
# # Maximum system capacity
# model.max_syscap = Param(model.m, within=NonNegativeReals, default=infinity)

In [101]:
##############################################
# Rules
############################################## 

# May not need???
# Limit amount of consumption 
def consumption_rule(model, d):
    value = sum(model.cons[d, h] for h in model.h)
    return model.min_cons[d] <= value <= model.max_cons[d]

# May not need???
# Limit amount of irradiance 
def irradiance_rule(model, d):
    value = sum(model.spi[d, h] for h in model.h)
    return model.min_spi[d] <= value <= model.max_spi[d]

# Set minimum system capacity (in W)
def min_system_capacity_rule(model):
    return model.x >= 1000

# # def o_rule(model):
# #     return summation(model.x)

# def energy_cost_savings(model):
#     return 0

# def solar_project_costs(model):
#     return 0

# # ROI = energy cost savings /  solar project costs
# def roi_rule(model):
#     return energy_cost_savings(model) / solar_project_costs(model)

# # def npv(model):
# #     return model

# # Objective function rules: Maximize ROI
# model.cost = Objective(rule = roi_rule)

In [102]:
##############################################
# Constraints
##############################################

model.consumption_constraint = Constraint(model.d, rule=consumption_rule)
model.irradiance_constraint = Constraint(model.d, rule=irradiance_rule)
model.min_syscap_constrait = Constraint(rule=min_system_capacity_rule)

model.c = ConstraintList()

## Generate Sets

In [106]:
# Generate .tab files

num_years = 10
num_months = 12
num_week_days = 7
num_month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
num_month_days_leap_year = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
num_hours = 24
daily_cons = 3600 #KWh
hourly_cons_pct = [0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 
                  0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417, 0.0417]
hourly_spi = [0, 0, 0, 0, 0, 0, 0, 0.1, 0.2, 0.4, 0.5, 0.65, 
              0.80, 0.85, 0.92, 0.87, 0.80, 0.70, 0.60, 0.40, 0.20, 0, 0, 0]
curr_year = datetime.datetime.now().year

# Generate .tab file for model.m
f = open("tab/m.tab","w+")
f.write("m")
for i in range(num_months):
    f.write("\n%d" % (i+1))
f.close()

# Generate .tab file for model.h
f = open("tab/h.tab","w+")
f.write("h")
for i in range(num_hours):
    f.write("\n%d" % i)
f.close()

# Generate .tab file for model.dw
f = open("tab/dw.tab","w+")
f.write("dw")
for i in range(num_week_days):
    f.write("\n%d" % (i+1))
f.close()

# Generate .tab file for model.d
f = open("tab/d.tab","w+")
f.write("d")
for i in range(num_years):
    y = curr_year + i
    #print "year = %d" % y
    if(isleap(y)):
        num_month_days_temp = num_month_days_leap_year
    else:
        num_month_days_temp = num_month_days
    for j in range(num_months):
        for k in range(num_month_days_temp[j]):
            f.write("\ny%d_m%d_d%d" % (y, j+1, k+1))
f.close()

# Generate .tab file for model.cons
f = open("tab/cons.tab","w+")
f.write("m\th\tdw\tcons")
for i in range(num_months):
    for j in range(num_week_days):
        for k in range(num_hours):
            f.write("\n%d\t%d\t%d\t%f" % (i+1, j+1, k, daily_cons * hourly_cons_pct[k]))
f.close()

# Generate .tab file for model.spi
f = open("tab/spi.tab","w+")
f.write("m\th\tspi")
for i in range(num_months):
    for j in range(num_hours):
        f.write("\n%d\t%d\t%f" % (i+1, j, hourly_spi[j]))
f.close()

In [107]:
# Create DataPortal object
data = DataPortal()

# Load sets
data.load(filename="tab/m.tab", set=model.m)
data.load(filename="tab/h.tab", set=model.h)
data.load(filename="tab/d.tab", set=model.d)
data.load(filename="tab/dw.tab", set=model.dw)

# print(data['m'])
# print(data['h'])
# print(data['d'])
# print(data['dw'])

# Load parameters
data.load(filename="tab/cons.tab", param=model.cons)
data.load(filename="tab/spi.tab", param=model.spi)

#print(data['cons'])
#print(data['spi'])

# instance = model.create(data)